# Bölüm 1: Temel Prompt Yapısı

- [Ders](#lesson)
- [Alıştırmalar](#exercises)
- [Örnek Oyun Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve.*yardımcı fonksiyonunu oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [ ]:
!pip install anthropic

# Python'un yerleşik düzenli ifade kütüphanesini içe aktar
import re
import anthropic

# API_KEY & MODEL_NAME değişkenlerini IPython store'dan al
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## Ders

Anthropic offers two APIs, the legacy [Text Completions API](https://docs.anthropic.com/claude/reference/complete_post) and the current [Messages API](https://docs.anthropic.com/claude/reference/messages_post). For this tutorial, we will be exclusively using the Messages API.

En azından, Messages API kullanarak Claude'a yapılan bir çağrı aşağıdaki parametreleri gerektirir:
- `model`: the [API model name](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations) of the model that you intend to call

- `max_tokens`: durmadan önce üretilecek maksimum token sayısı. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

- `messages`: giriş mesajlarının bir dizisi. Our models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the user `role`.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit our [API documentation](https://docs.anthropic.com/claude/reference/messages_post).

### Örnekler

Claude'un doğru biçimlendirilmiş bazı prompt'lara nasıl yanıt verdiğine bakalım. Aşağıdaki hücrelerin her biri için, hücreyi çalıştırın (`shift+enter`), ve Claude'un yanıtı bloğun altında görünecek.

In [ ]:
# Prompt
PROMPT = "Merhaba Claude, nasılsın?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Bana okyanusun rengini söyleyebilir misin?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Celine Dion hangi yılda doğdu?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

Now let's take a look at some prompts that do not include the correct Messages API formatting. For these malformatted prompts, the Messages API returns an error.

First, we have an example of a Messages API call that lacks `role` and `content` fields in the `messages` array.

In [ ]:
# Claude'un yanıtını al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Merhaba Claude, nasılsın?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

Here's a prompt that fails to alternate between the `user` and `assistant` roles.

In [ ]:
# Claude'un yanıtını al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "Celine Dion hangi yılda doğdu?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompt'lar

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Kurulum](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Örneği

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Örnek Oyun Alanı**](#example-playground).

---

## Alıştırmalar
- [Alıştırma 1.1 - Üçe Kadar Sayma](#exercise-11---counting-to-three)
- [Alıştırma 1.2 - System Prompt](#exercise-12---system-prompt)

### Alıştırma 1.1 - Üçe Kadar Sayma
Uygun.*biçimlendirmesini kullanarak, Claude'un.*üçe kadar saymasını sağlamak için aşağıdaki.*düzenleyin.** Çıktı ayrıca çözümünüzün doğru olup olmadığını da gösterecek.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Claude'un yanıtını al
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Claude'un yanıtını yazdır and the corresponding grade
print(response)
print("\n--------------------------- PUANLAMA ---------------------------")
print("Bu alıştırma doğru şekilde çözüldü:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Alıştırma 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Claude'un yanıtını al
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Claude'un yanıtını yazdır and the corresponding grade
print(response)
print("\n--------------------------- PUANLAMA ---------------------------")
print("Bu alıştırma doğru şekilde çözüldü:", grade_exercise(response))

❓ Bir ipucu istiyorsanız, aşağıdaki hücreyi çalıştırın!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız. İyi prompt'lama!

---

## Örnek Oyun Alanı

Bu, prompt örnekleriyle özgürce deney yapabileceğiniz bir alan bu derste gösterilen ve Claude'un yanıtlarını nasıl etkileyebileceğini görmek için prompt'ları değiştirin.

In [ ]:
# Prompt
PROMPT = "Merhaba Claude, nasılsın?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Bana okyanusun rengini söyleyebilir misin?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Celine Dion hangi yılda doğdu?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Claude'un yanıtını al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Merhaba Claude, nasılsın?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

In [ ]:
# Claude'un yanıtını al
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "Celine Dion hangi yılda doğdu?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Claude'un yanıtını yazdır
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))